# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [9]:
!pip install pathway bokeh pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 

In [10]:
import pandas as pd

# Load your CSV
df = pd.read_csv("Parking_stream(1).csv", parse_dates=["Timestamp"])

# Preview
df.head()


,SystemCodeNumber_Encoded,QueueLength_Normalized,VehicleTypeWeight,Traffic,IsSpecialDay,Occ_Cap_Norm,Timestamp
0,0,0.066667,0.66,0.0,0,0.098521,2016-10-04 07:59:00
1,4,0.133333,0.33,0.0,0,0.186953,2016-10-04 07:59:00
2,3,0.133333,0.66,0.0,0,0.366915,2016-10-04 07:59:00
3,5,0.133333,0.66,0.0,0,0.491326,2016-10-04 07:59:00
4,13,0.133333,0.00,0.0,0,0.304781,2016-10-04 07:59:00


In [11]:
import pathway as pw
import datetime

base_price = 10.0
alpha = 2.0

# Assuming df contains your data loaded already
data = pw.debug.table_from_pandas(df)

tumbling_window = (
    data.windowby(
        pw.this.Timestamp,
        instance=pw.this.SystemCodeNumber_Encoded,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        LotID=pw.reducers.max(pw.this.SystemCodeNumber_Encoded),
        occ_avg=pw.reducers.max(pw.this.Occ_Cap_Norm),
    )
    .with_columns(
        Price=base_price + alpha * pw.this.occ_avg
    )
)

pw.debug.compute_and_print(tumbling_window, include_id=False)


t                   | LotID | occ_avg            | Price
2016-10-05 00:00:00 | 0     | 0.4458506107690383 | 10.891701221538076
2016-10-05 00:00:00 | 1     | 0.8282123085818337 | 11.656424617163667
2016-10-05 00:00:00 | 2     | 0.9478651107327292 | 11.895730221465458
2016-10-05 00:00:00 | 3     | 0.9096758344217376 | 11.819351668843476
2016-10-05 00:00:00 | 4     | 0.6221355449889324 | 11.244271089977865
2016-10-05 00:00:00 | 5     | 0.7535579414431994 | 11.5071158828864
2016-10-05 00:00:00 | 6     | 0.9503904858368332 | 11.900780971673667
2016-10-05 00:00:00 | 7     | 0.7083781839920972 | 11.416756367984194
2016-10-05 00:00:00 | 8     | 0.2046233529035083 | 10.409246705807016
2016-10-05 00:00:00 | 9     | 0.8033443945169817 | 11.606688789033964
2016-10-05 00:00:00 | 10    | 0.4549566227663972 | 10.909913245532794
2016-10-05 00:00:00 | 11    | 0.6059568744525353 | 11.21191374890507
2016-10-05 00:00:00 | 12    | 0.3993885666283828 | 10.798777133256765
2016-10-05 00:00:00 | 13    | 0.7293

In [12]:
price_df = pw.debug.compute_and_print(tumbling_window, include_id=False)


t                   | LotID | occ_avg            | Price
2016-10-05 00:00:00 | 0     | 0.4458506107690383 | 10.891701221538076
2016-10-05 00:00:00 | 1     | 0.8282123085818337 | 11.656424617163667
2016-10-05 00:00:00 | 2     | 0.9478651107327292 | 11.895730221465458
2016-10-05 00:00:00 | 3     | 0.9096758344217376 | 11.819351668843476
2016-10-05 00:00:00 | 4     | 0.6221355449889324 | 11.244271089977865
2016-10-05 00:00:00 | 5     | 0.7535579414431994 | 11.5071158828864
2016-10-05 00:00:00 | 6     | 0.9503904858368332 | 11.900780971673667
2016-10-05 00:00:00 | 7     | 0.7083781839920972 | 11.416756367984194
2016-10-05 00:00:00 | 8     | 0.2046233529035083 | 10.409246705807016
2016-10-05 00:00:00 | 9     | 0.8033443945169817 | 11.606688789033964
2016-10-05 00:00:00 | 10    | 0.4549566227663972 | 10.909913245532794
2016-10-05 00:00:00 | 11    | 0.6059568744525353 | 11.21191374890507
2016-10-05 00:00:00 | 12    | 0.3993885666283828 | 10.798777133256765
2016-10-05 00:00:00 | 13    | 0.7293

In [13]:

# Sink the results to CSV
pw.io.csv.write(tumbling_window, "price_output.csv")

# Execute the pipeline to flush data
pw.run()



Output()

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [14]:
import pandas as pd

price_df = pd.read_csv("price_output.csv")
price_df.head()


,t,LotID,occ_avg,Price,time,diff
0,2016-10-24T00:00:00.000000000,1,0.855599,11.711198,0,1
1,2016-10-18T00:00:00.000000000,6,0.918274,11.836547,0,1
2,2016-10-17T00:00:00.000000000,11,0.586279,11.172557,0,1
3,2016-12-02T00:00:00.000000000,9,0.847016,11.694032,0,1
4,2016-11-04T00:00:00.000000000,5,0.682040,11.364080,0,1


In [30]:
pw.io.csv.write(tumbling_window, "price_output.csv")
pw.run()

import pandas as pd
price_df = pd.read_csv("price_output.csv")
print(price_df.head())


Output()

                               t  LotID   occ_avg      Price  time  diff
0  2016-10-24T00:00:00.000000000      1  0.855599  11.711198     0     1
1  2016-10-18T00:00:00.000000000      6  0.918274  11.836547     0     1
2  2016-10-17T00:00:00.000000000     11  0.586279  11.172557     0     1
3  2016-12-02T00:00:00.000000000      9  0.847016  11.694032     0     1
4  2016-11-04T00:00:00.000000000      5  0.682040  11.364080     0     1


In [17]:
print(price_df["LotID"].unique())


[ 1  6 11  9  5  4  0 13  2 10  7  8  3 12]


In [18]:
lot_id_demo = price_df["LotID"].unique()[0]


In [19]:
filtered_df = price_df[price_df["LotID"] == lot_id_demo].sort_values(by="t")
print(filtered_df.head())


                                 t  LotID   occ_avg      Price  time  diff
952  2016-10-05T00:00:00.000000000      1  0.828212  11.656425     0     1
508  2016-10-06T00:00:00.000000000      1  0.855599  11.711198     0     1
6    2016-10-07T00:00:00.000000000      1  0.818254  11.636507     0     1
798  2016-10-08T00:00:00.000000000      1  0.708708  11.417416     0     1
954  2016-10-09T00:00:00.000000000      1  0.855599  11.711198     0     1


In [20]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

output_notebook()

# Step 1: Convert 't' to datetime
price_df["t"] = pd.to_datetime(price_df["t"])

# Step 2: Loop over unique LotIDs
lot_ids = price_df["LotID"].unique()

for lot_id in lot_ids:
    filtered_df = price_df[price_df["LotID"] == lot_id].sort_values(by="t")

    source = ColumnDataSource(data=dict(t=filtered_df["t"], price=filtered_df["Price"]))

    p = figure(x_axis_type="datetime", title=f"Price Evolution for Lot {lot_id}",
               width=700, height=400)
    p.line(x='t', y='price', source=source, line_width=2, color='blue')
    p.circle(x='t', y='price', source=source, size=5, color='red')

    show(p)
